Model Training

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
init_df = pd.read_csv("data\gemstone.csv")
init_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
init_df=init_df.drop(columns=['id'])
init_df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
# Seggregation of Independent and Dependent feature
x=init_df.drop(columns=['price'],axis=1)
y=init_df['price']

In [5]:
# Seggregation between categorical and numerical feature
cat_feature=x.select_dtypes(include='object').columns
num_feature=x.select_dtypes(exclude='object').columns

In [6]:
x['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [7]:
# Defining rank to categorical features
cut_rank=['Fair','Good','Very Good','Premium','Ideal']
color_rank=['D','E','F','G','H','I','J']
clarity_rank=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [8]:
from sklearn.impute import SimpleImputer #Handling Missing values
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OrdinalEncoder

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [9]:
# Numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())

    ]
)

# Categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_rank,color_rank,clarity_rank])),
        ('scaler',StandardScaler())

    ]
)


preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_feature),
    ('cat_pipeline',cat_pipeline,cat_feature)
])

In [10]:
# Train Test Split
from sklearn.model_selection import train_test_split

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.25)

In [12]:
# Applying the pipelines to train_set and test_test
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [13]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.472019,0.257973,-0.640440,0.606460,0.624162,0.644271,0.873371,1.527634,-1.315033
1,0.623099,0.812463,-1.161268,0.732515,0.787278,0.803549,0.873371,0.911526,0.017494
2,2.651895,-1.867571,1.442873,2.236173,2.318751,2.005371,-0.132110,1.527634,0.017494
3,-1.017203,1.089708,-0.640440,-1.248351,-1.233542,-1.151223,-1.137590,0.295418,-0.648770
4,-0.110720,1.459368,3.526185,0.039212,-0.010176,0.137478,-2.143070,-0.936798,-1.315033


In [14]:
# Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [15]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2=r2_score(true,predicted)

    return mae,mse,rmse,r2



In [16]:
# Multiple model training

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'RFR':RandomForestRegressor(),
    'KNR':KNeighborsRegressor(),
    'SVR':SVR()
    
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)


    y_pred=model.predict(x_test)

    mae,mse,rmse,r2=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    
    print("RMSE: ",rmse)
    print("MAE: ",mae)
    print("MSE: ",mse)
    print("R2: ",r2)

    r2_list.append(r2)

    print("="*30)
    print("\n")




LinearRegression
RMSE:  1018.1581640245458
MAE:  675.8646403365957
MSE:  1036646.0469698339
R2:  0.9361253831282493


Lasso
RMSE:  1018.2756930699593
MAE:  677.07214907871
MSE:  1036885.387097106
R2:  0.9361106358005803


Ridge
RMSE:  1018.1611429115582
MAE:  675.8935091357502
MSE:  1036652.1129349703
R2:  0.9361250093640314


ElasticNet
RMSE:  1538.6866758139975
MAE:  1062.8025492504894
MSE:  2367556.6863275296
R2:  0.8541191791514899


RFR
RMSE:  611.4870575691361
MAE:  309.6681122480256
MSE:  373916.42157456
R2:  0.9769605370705416


KNR
RMSE:  678.4935001969293
MAE:  351.5141422490391
MSE:  460353.42980948056
R2:  0.971634581503852


SVR
RMSE:  1863.0791053464877
MAE:  820.9060646262831
MSE:  3471063.7527786694
R2:  0.7861248128093135


